<a href="https://colab.research.google.com/github/SilvioRFernandes/Prontuario_Inteligente/blob/main/RAG_prontuarios_com_Gemini_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sistema de Prontuário Inteligente

## Autor: Silvio Fernandes <silvio@ufersa.edu.br>

#### Descricao das funcionalidades do sistema:

1. Cadastrar paciente (dados pessoais)
2. Cadastrar consulta de um paciente (data e texto prontuário médico)
3. Anonimizar paciente (tornar anonimo - sem identificação do paciente)
4. Fazer resumo da história clínica do paciente
5. Destacar para o médico o que não pode esquecer de perguntar na consulta atual
6. Sugerir diagnótico da doença
7. Buscar informações de um paciente específico

# Configuração dos Modelos

Instalando SDK para API do Gemini

In [ ]:
#!pip install -q -U google-generativeai

Setando a API Key

In [ ]:
# Importando bibliotecas
import google.generativeai as genai
from google.colab import userdata
import pandas as pd
import numpy as np

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Configurando os modelos

In [ ]:
version_model_gen = 'gemini-1.5-pro-latest' #generative
version_model_emb = 'models/embedding-001' #embedding

gen_config = {
    'candidate_count': 1,
    'temperature': 0
}

safety_config = {
  'Dangerous': 'Block_None' #Necessario por tratar de drogas
}

instrucao_sistema ="Você é um médico"

title = "The next generation of AI for developers and Google Workspace"
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications")


Inicializando os modelos

In [ ]:
# model_gen = genai.GenerativeModel(
#                               model_name=version_model_gen,
#                               generation_config= gen_config,
#                               safety_settings = safety_config,
#                               system_instruction=instrucao_sistema #só está disponível no gemini 1.5
#                               )

# model_retrival = genai.embed_content(model=version_model_emb,
#                                 content=sample_text,
#                                 task_type="retrieval_document",
#                                 title=title)

# Descricao das funcionalidades do sistema:

1. Cadastrar paciente (dados pessoais)
2. Cadastrar consulta de um paciente (data e texto prontuário médico)
3. Anonimizar paciente
4. Fazer resumo da história clínica do paciente
5. Destacar para o médico o que não pode esquecer de perguntar na consulta atual (algum sintoma ou resultado de um exame solicitado antes)
6. Sugerir diagnótico da doença
7. Quantificar nível da doença
8. Sugerir conduta e prazo para retorno do paciente
9. Avaliar a evolução da doença (com base nas consultas até o momento) - mostrar em forma de gráfico
  * Quando houver piora, tentar inferir acontecimentos que podem ter causado
10. Avaliar a evolução de um tratamento específico (ex: um tipo de medicamento) - mostrar em forma de gráfico
  * Quando houver piora, tentar inferir decisões que a causaram e buscar alternativas
11. Buscar informações de um paciente específico
12. Buscar informações entre todos os pacientes

# Bancos de Dados

In [ ]:
df_pacientes = pd.DataFrame(columns = ['ID_paciente', 'Nome', 'Data_nascimento', 'Endereco', 'Doencas', 'Primeira_consulta'])
#df_pacientes

df_prontuarios = pd.DataFrame(columns = ['ID_paciente', 'ID_doenca', 'Data', 'Texto', 'Texto_Anonimo', 'Classificacao_doenca', 'Classificacao_tratamento', 'Embeddings'])
#df_prontuarios

# Funcoes

## Manipulação dos Dataframes

In [ ]:
def cadastrar_paciente(df, nome, data_nascimento, endereco):
  paciente ={
      'ID_paciente': len(df_pacientes),
      'Nome': nome,
      'Data_nascimento': data_nascimento,
      'Endereco': endereco,
      'Doencas': '',
      'Primeira_consulta': True
  }
  print('>>> SISTEMA: Paciente cadastrado com sucesso')
  return pd.concat([df, pd.DataFrame([paciente])], ignore_index=True)

In [ ]:
def cadastrar_consulta(df, ID_paciente, Data, Texto):
  #columns = ['ID_paciente', 'ID_doenca', 'Data', 'Texto', 'Texto_Anonimo', 'Classificacao_doenca', 'Classificacao_tratamento', 'Embeddings']

  #Se for a primeira consultada do paciente - fazer a classificacao da doenca
  indice=df_pacientes.query(f'ID_paciente=={ID_paciente} and Primeira_consulta==True')
  if len(indice) > 0: #Paciente existe e é a primeira consulta
    #Desligando a flag
    df_pacientes.loc[indice.index, 'Primeira_consulta'] = False

    #Chamando o Gemini para diagnostico
    tarefa = "Qual o diagnóstico mais provável? Apenas o nome da doença"
    print(f'>>> SISTEMA: identificando diagnóstico mais provável...')
    doenca = llm_executa(Texto, tarefa)
    print(f'>>> SISTEMA: o diagnóstico mais provável é {doenca} e será guardado no registro do paciente')
    df_pacientes.loc[indice.index, 'ID_doenca'] = doenca

  else:
    print('>>> SISTEMA: Não é primeira consulta') #so para teste
    doenca = df_pacientes.loc[ID_paciente, 'ID_doenca']

  #Chamando o Gemini para diagnostico
  tarefa = "Anonimize o texto, sem alterar nada das demais informações: "
  anonimo = llm_executa(Texto, tarefa)
  print(f'>>> SISTEMA: anonimizando paciente: \n {anonimo}')

  #Calculando embeddings dessa consulta - Titulo é a data
  emb = embed_fn(Data, Texto)

  dados_consulta = {
    "ID_paciente": ID_paciente,
    'ID_doenca': doenca,
    "Data": Data,
    "Texto": Texto,
    'Texto_Anonimo': anonimo,
    "Classificacao_doenca": '',
    "Classificacao_tratamento": '',
    'Embeddings': emb
  }

  df = pd.concat([df, pd.DataFrame([dados_consulta])], ignore_index=True)
  print('>>> SISTEMA: Consulta cadastrada com sucesso')
  return df

## Interacao com LLM

In [ ]:
def llm_executa(texto, tarefa):
  #Um modelo para cada execução para não relacionar com consultas anteriores
  model_gen = genai.GenerativeModel(
                              model_name=version_model_gen,
                              generation_config= gen_config,
                              safety_settings = safety_config,
                              system_instruction=instrucao_sistema #só está disponível no gemini 1.5
                              )
  response = model_gen.generate_content(Texto+tarefa)
  return (response.text).rstrip() #remove \n no final da string

In [ ]:
def llm_analisa_doenca(ID_paciente):
  json_data = (df_prontuarios.query(f'ID_paciente=={ID_paciente}')).to_json()
  tarefa = "Como avalia a evolução da doença de forma bem sucinta?"
  response = llm_executa(json_data, tarefa)
  return response

In [ ]:
def llm_historia_clinica(ID_paciente):
  json_data = (df_prontuarios.query(f'ID_paciente=={ID_paciente}')).to_json()
  tarefa = "Resuma a história clínica desse paciente"
  response = llm_executa(f'{json_data}', tarefa)
  return response


In [ ]:
def llm_lembrar_medico(ID_paciente):
  resumo = llm_historia_clinica(ID_paciente)
  contexto = f"Considere a história clínica: {resumo}"
  tarefa = "O que o médico não pode esquecer?"
  response = llm_executa(contexto, tarefa)
  return response

In [ ]:
def llm_lembrar_paciente(ID_paciente):
  resumo = llm_historia_clinica(ID_paciente)
  contexto = f"Considere a história clínica: {resumo}"
  tarefa = "O que o paciente não pode esquecer?"
  response = llm_executa(contexto, tarefa)
  return response

In [ ]:
def llm_animizar_paciente(texto):
  tarefa = "Anonimize o texto a seguir, sem alterar nada das demais informações: "
  config = {
      'candidate_count': 1,
      'temperature': 0
  }

  model = genai.GenerativeModel(
      model_name=version_model_gen,
      generation_config= config
    )

  response = model.generate_content(tarefa+texto)
  return response.text

In [ ]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(titulo, texto):
  return genai.embed_content(model=version_model_emb,
                             content=texto,
                             task_type="retrieval_document",
                             title=titulo)["embedding"]

In [ ]:
def criar_embeddings(df):
  df['Embeddings'] = df.apply(lambda row: embed_fn(row['Data'], row['Texto']), axis=1)
  return df

In [ ]:
#Busca informacoes nos registros de 1 unico paciente
def buscar_info_paciente(query, ID_paciente):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  paciente = df_prontuarios.query(f'ID_paciente=={ID_paciente}')
  query_embedding = genai.embed_content(model=version_model_emb,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(paciente['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return paciente.iloc[idx] # Return text from index with max value

In [ ]:
#Busca informacoes nos registros de todos os pacientes
def buscar_info(query):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=version_model_emb,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(df_prontuarios['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products) #mudar para escolher apenas valores a partir de um valor
  return df_prontuarios.iloc[idx] # Return text from index with max value

## Testando o cadastro

In [ ]:
# @title Cadastro de Pacientes { display-mode: "both" }
nome = "Ana Silva" # @param {type:"string"}
nascimento = "1978-07-15" # @param {type:"date"}
endereco = "Rua das Flores, 123, Cidade Feliz"

df_pacientes = cadastrar_paciente(df_pacientes, nome, nascimento, endereco)
df_pacientes

>>> SISTEMA: Paciente cadastrado com sucesso


,ID_paciente,Nome,Data_nascimento,Endereco,Doencas,Primeira_consulta
0,0,Ana Silva,1978-07-15,"Rua das Flores, 123, Cidade Feliz",,True


In [ ]:
# @title Cadastro de Pacientes { display-mode: "both" }
nome = "Maria das Graças" # @param {type:"string"}
nascimento = "1992-03-07" # @param {type:"date"}
endereco = "Av. Tom Jobim, 53, Cidade da Musica" # @param {type:"string"}

df_pacientes = cadastrar_paciente(df_pacientes, nome, nascimento, endereco)
df_pacientes

>>> SISTEMA: Paciente cadastrado com sucesso


,ID_paciente,Nome,Data_nascimento,Endereco,Doencas,Primeira_consulta
0,0,Ana Silva,1978-07-15,"Rua das Flores, 123, Cidade Feliz",,True
1,1,Maria das Graças,1992-03-07,"Av. Tom Jobim, 53, Cidade da Musica",,True


In [ ]:
#Primeira consulta da 1a paciente
# @title Cadastro de consulta de um paciente
ID_paciente = 0 # @param {type:"number"}
Data = "2024-05-10" # @param {type:"date"}
Texto = "Ana Silva, nascida em 15 de julho de 1978 e residente na Rua das Flores, 123, Cidade Feliz, começou seus sintomas em 10 de março de 2015 . Seus sintomas atuais incluem dor nas articulações, rigidez matinal e fadiga, com um nível de dor avaliado em 6/ 10 na escala visual analógica. No histórico de tratamentos, Ana já utilizou metotrexato, sulfassalazina, e passou por fisioterapia. Em 22 de setembro de 2018, realizou uma infiltração de corticosteroides nas articulações. Atualmente, está em tratamento com metotrexato (15 mg/semana), hidroxicloroquina (200 mg/dia), prednisona (5 mg/dia) e naproxeno (500 mg quando necessário). O histórico familiar revela que sua mãe tem histórico de artrite reumatoid e, enquanto o pai não apresenta doenças autoimunes. Exames médicos indicam resultados como fator reumatoide e anti-CCP positiv os, além de erosões articulares em mãos e punhos em raios-X. A avaliação funcional demonstra uma dificuldade moderada em realizar atividades diárias, com um escore de Avaliação Funcional (HAQ-DI) de 1,5. Exames laboratoriais recentes: Hemoglobina de 12.5 g/dL, Contagem de Leucócitos de 8,000/mm³, Plaquetas de 250,000/mm³, Velocidade de Hemossedimentação (VHS): 20 mm/h, Proteína C Reativa (PCR): 15 mg/L, Dosagem do Fator Reumatoide (FR): Positivo (150 UI/mL), Anti-CCP: Positivo (80 U/mL), AST (TGO): 25 U/L, ALT (TGP): 30 U/L, Creatinina: 0.8 mg/dL, Ureia: 20 mg/dL, Níveis de Vitamina D: 30 ng/mL, Colesterol Total de 180 mg/dL e Triglicerídeos  de 120 mg/dL. As consultas anteriores registradas incluem ajustes na terapia medicamentosa em 30 de abril de 2022 e uma avaliação de resposta ao tratamento em 15 de dezembro de 2022. Os planos de tratamento consistem em manter a terapia medicamentosa atual, realizar acompanhamento regular com o reumatologista e incorporar sessões de fisioterapia pa ra melhorar a mobilidade de Ana Silva." # @param {type:"string"}

df_prontuarios = cadastrar_consulta(df_prontuarios, ID_paciente, Data, Texto)
df_prontuarios

>>> SISTEMA: identificando diagnóstico mais provável...
>>> SISTEMA: o diagnóstico mais provável é Artrite Reumatoide e será guardado no registro do paciente
>>> SISTEMA: anonimizando paciente: 
 Uma paciente do sexo feminino, nascida em 15 de julho de 1978, começou a apresentar sintomas em 10 de março de 2015. Seus sintomas atuais incluem dor nas articulações, rigidez matinal e fadiga, com um nível de dor avaliado em 6/10 na escala visual analógica. No histórico de tratamentos, a paciente já utilizou metotrexato, sulfassalazina, e passou por fisioterapia. Em 22 de setembro de 2018, realizou uma infiltração de corticosteroides nas articulações. Atualmente, está em tratamento com metotrexato (15 mg/semana), hidroxicloroquina (200 mg/dia), prednisona (5 mg/dia) e naproxeno (500 mg quando necessário). O histórico familiar revela que sua mãe tem histórico de artrite reumatoide e, enquanto o pai não apresenta doenças autoimunes. Exames médicos indicam resultados como fator reumatoide e anti

,ID_paciente,ID_doenca,Data,Texto,Texto_Anonimo,Classificacao_doenca,Classificacao_tratamento,Embeddings
0,0,Artrite Reumatoide,2024-05-10,"Ana Silva, nascida em 15 de julho de 1978 e re...","Uma paciente do sexo feminino, nascida em 15 d...",,,"[0.050345007, -0.019977478, -0.022784408, 0.06..."


In [ ]:
#Segunda consulta da 1a paciente
ID_paciente = 0 # @param {type:"number"}
Data = "2024-07-18" # @param {type:"date"}
Texto = "Ana Silva, diagnosticada com artrite reumatoide, compareceu para sua consulta de retorno após 2 meses em tratamento com metotrexato (15 mg/semana), hidroxicloroquina (200 mg/dia) e prednisona (5 mg/dia). Relatou melhora nos sintomas da doença , incluindo diminuição da dor nas articulações e rigidez matinal. Contudo, Ana observou alguns problemas durantes esses dois meses. Reportou ganho de peso e aumento da pressão arteria l, além de dificuldades visuais, como visão turva e dificuldade de adaptação à lu z. Esses sintomas requerem uma avaliação oftalmológica urgente para excluir toxicidade ocular. Ana também trouxe os resultados dos exames laboratoriais mais recentes. O AST (TGO) estava elevado a 45 U/L e o ALT (TGP) a 50 U/L, ambos acima dos valores de referência normais (0 a 40 U/L ). Adicionalmente, a hemoglobina diminuiu para 11.0 g/dL, indicativo de uma possível anemia." # @param {type:"string"}

df_prontuarios = cadastrar_consulta(df_prontuarios, ID_paciente, Data, Texto)
df_prontuarios

>>> SISTEMA: Não é primeira consulta
>>> SISTEMA: anonimizando paciente: 
 Paciente diagnosticada com artrite reumatoide compareceu para consulta de retorno após 2 meses em tratamento com metotrexato (15 mg/semana), hidroxicloroquina (200 mg/dia) e prednisona (5 mg/dia). Relatou melhora nos sintomas da doença, incluindo diminuição da dor nas articulações e rigidez matinal. Contudo, a paciente observou alguns problemas durantes esses dois meses. Reportou ganho de peso e aumento da pressão arterial, além de dificuldades visuais, como visão turva e dificuldade de adaptação à luz. Esses sintomas requerem uma avaliação oftalmológica urgente para excluir toxicidade ocular. A paciente também trouxe os resultados dos exames laboratoriais mais recentes. O AST (TGO) estava elevado a 45 U/L e o ALT (TGP) a 50 U/L, ambos acima dos valores de referência normais (0 a 40 U/L). Adicionalmente, a hemoglobina diminuiu para 11.0 g/dL, indicativo de uma possível anemia.
>>> SISTEMA: Consulta cadastrada co

,ID_paciente,ID_doenca,Data,Texto,Texto_Anonimo,Classificacao_doenca,Classificacao_tratamento,Embeddings
0,0,Artrite Reumatoide,2024-05-10,"Ana Silva, nascida em 15 de julho de 1978 e re...","Uma paciente do sexo feminino, nascida em 15 d...",,,"[0.050345007, -0.019977478, -0.022784408, 0.06..."
1,0,Artrite Reumatoide,2024-07-18,"Ana Silva, diagnosticada com artrite reumatoid...",Paciente diagnosticada com artrite reumatoide ...,,,"[0.04399745, -0.041109204, -0.038250357, 0.051..."


In [ ]:
#Primeira consulta da 2a paciente
ID_paciente = 1 # @param {type:"number"}
Data = "2024-03-12" # @param {type:"date"}
Texto = "Maria das Gracas, 32 anos, solteira, contadora, encaminhada para avaliação médica devido a sintomas consistentes de depressão. A paciente relata sentimentos persistentes de tristeza, falta de interesse em atividades prazerosas, alterações no sono, apetite reduzido, fadiga constante e dificuldade de concentração , presentes há cerca de seis meses e impactando significativamente suas atividades diárias. No histórico médico, destacam-se episódios de ansiedade e estresse no trabalho. Não há relato de doenças psiquiátricas na família, e a paciente não fuma, consumindo álcool de forma moderada. Os exames complementares mostram: Leucócitos: 8.000/mm³; Hemácias: 4. 5 milhões/mm³; Plaquetas: 250.000/mm³; Colesterol Total: 180 mg/dL; HDL: 50 mg/dL; LDL: 110 mg/dL; Triglicerídeos: 80 mg/dL; Glicose: 90 mg/dL; Hemoglobina Glicada (A1c): 5.5%; TSH: 2.5 uIU/mL; T3 Livre: 2,5 pg/mL; T4 Livre: 1,2 ng/dL. A avaliação psicológica evidenciou sentimentos de desesperança, baixa autoestima e isolamento social, além de pensamentos recorrentes de autocrítica e dificuldade em lidar com atividades cotidianas." # @param {type:"string"}

df_prontuarios = cadastrar_consulta(df_prontuarios, ID_paciente, Data, Texto)
df_prontuarios


>>> SISTEMA: identificando diagnóstico mais provável...
>>> SISTEMA: o diagnóstico mais provável é Depressão. e será guardado no registro do paciente
>>> SISTEMA: anonimizando paciente: 
 Paciente do sexo feminino, 32 anos, encaminhada para avaliação médica devido a sintomas consistentes de depressão. A paciente relata sentimentos persistentes de tristeza, falta de interesse em atividades prazerosas, alterações no sono, apetite reduzido, fadiga constante e dificuldade de concentração, presentes há cerca de seis meses e impactando significativamente suas atividades diárias. No histórico médico, destacam-se episódios de ansiedade e estresse no trabalho. Não há relato de doenças psiquiátricas na família, e a paciente não fuma, consumindo álcool de forma moderada. Os exames complementares mostram: Leucócitos: 8.000/mm³; Hemácias: 4. 5 milhões/mm³; Plaquetas: 250.000/mm³; Colesterol Total: 180 mg/dL; HDL: 50 mg/dL; LDL: 110 mg/dL; Triglicerídeos: 80 mg/dL; Glicose: 90 mg/dL; Hemoglobina Gli

,ID_paciente,ID_doenca,Data,Texto,Texto_Anonimo,Classificacao_doenca,Classificacao_tratamento,Embeddings
0,0,Artrite Reumatoide,2024-05-10,"Ana Silva, nascida em 15 de julho de 1978 e re...","Uma paciente do sexo feminino, nascida em 15 d...",,,"[0.050345007, -0.019977478, -0.022784408, 0.06..."
1,0,Artrite Reumatoide,2024-07-18,"Ana Silva, diagnosticada com artrite reumatoid...",Paciente diagnosticada com artrite reumatoide ...,,,"[0.04399745, -0.041109204, -0.038250357, 0.051..."
2,1,Depressão.,2024-03-12,"Maria das Gracas, 32 anos, solteira, contadora...","Paciente do sexo feminino, 32 anos, encaminhad...",,,"[0.071816355, -0.03363534, -0.038312245, 0.044..."


In [ ]:
df_prontuarios

,ID_paciente,ID_doenca,Data,Texto,Texto_Anonimo,Classificacao_doenca,Classificacao_tratamento,Embeddings
0,0,Artrite Reumatoide,2024-05-10,"Ana Silva, nascida em 15 de julho de 1978 e re...","Uma paciente do sexo feminino, nascida em 15 d...",,,"[0.050345007, -0.019977478, -0.022784408, 0.06..."
1,0,Artrite Reumatoide,2024-07-18,"Ana Silva, diagnosticada com artrite reumatoid...",Paciente diagnosticada com artrite reumatoide ...,,,"[0.04399745, -0.041109204, -0.038250357, 0.051..."


In [ ]:
#Terceira consulta da 1a paciente
ID_paciente = 0 # @param {type:"number"}
Data = "2024-10-05" # @param {type:"date"}
Texto = "Ana Silva retorna para avaliação de seguimento após a implementação das alterações no tratamento para sua artrite reumatoide, estabelecidas na última consulta. O plano incluía ajustes na medicação devido a preocupações com hepatotoxicidade, efeitos colaterais da prednisona e potenciais problemas visuais associados ao uso de hidroxicloroquina. A paciente relata que, apesar das mudanças no tratamento, continua enfrentando desafios significativos: A avaliação oftalmológica não identificou toxicidade ocular grave; contudo, Ana continua a reportar desconforto visual leve, possivelmente relacionado ao uso contínuo da hidroxicloroquina, os esforços para gerenciar o ganho de peso e o aumento da pressão arterial devido ao uso de prednisona tiveram sucesso limitado. Também foi observado pelos seus novos exames que a função hepática melhorou após a redução da dose de metotrexato, mas os níveis de AST e ALT, embora menores, ainda estão ligeiramente acima do normal" # @param {type:"string"}

df_prontuarios = cadastrar_consulta(df_prontuarios, ID_paciente, Data, Texto)
df_prontuarios

# Embeddings

In [ ]:
ID_paciente = 0
query = "Quando a paciente foi diagnosticada?"
resposta = buscar_info_paciente(query, ID_paciente)
print(f"Em {resposta['Data']}: \n {resposta['Texto']}")

Em 2024-05-10: 
 Ana Silva, nascida em 15 de julho de 1978 e residente na Rua das Flores, 123, Cidade Feliz, começou seus sintomas em 10 de março de 2015 . Seus sintomas atuais incluem dor nas articulações, rigidez matinal e fadiga, com um nível de dor avaliado em 6/ 10 na escala visual analógica. No histórico de tratamentos, Ana já utilizou metotrexato, sulfassalazina, e passou por fisioterapia. Em 22 de setembro de 2018, realizou uma infiltração de corticosteroides nas articulações. Atualmente, está em tratamento com metotrexato (15 mg/semana), hidroxicloroquina (200 mg/dia), prednisona (5 mg/dia) e naproxeno (500 mg quando necessário). O histórico familiar revela que sua mãe tem histórico de artrite reumatoid e, enquanto o pai não apresenta doenças autoimunes. Exames médicos indicam resultados como fator reumatoide e anti-CCP positiv os, além de erosões articulares em mãos e punhos em raios-X. A avaliação funcional demonstra uma dificuldade moderada em realizar atividades diárias, c

In [ ]:
#Resumo da história clinica para o médico antes de uma nova consulta
ID_paciente = 0
resumo = llm_historia_clinica(ID_paciente)
resumo

'## Resumo da História Clínica de Ana Silva:\n\nAna Silva, paciente com artrite reumatoide, retornou para acompanhamento após 2 meses de tratamento com metotrexato, hidroxicloroquina e prednisona. \n\n**Pontos Positivos:**\n\n* Relata melhora dos sintomas da artrite reumatoide, com diminuição da dor articular e rigidez matinal.\n\n**Pontos Negativos:**\n\n* Ganho de peso.\n* Aumento da pressão arterial.\n* Dificuldades visuais, incluindo visão turva e dificuldade de adaptação à luz, **sugerindo possível toxicidade ocular pela hidroxicloroquina e necessitando avaliação oftalmológica urgente.**\n* Elevação das enzimas hepáticas AST (TGO) e ALT (TGP), indicando possível hepatotoxicidade pelo metotrexato.\n* Diminuição da hemoglobina, sugerindo anemia, possivelmente relacionada ao metotrexato.\n\n**Conduta:**\n\nA situação de Ana exige atenção médica imediata. É crucial investigar a causa da elevação das enzimas hepáticas, da anemia e dos problemas visuais, além de controlar a pressão arte

In [ ]:
#Destaques para o medico
ID_paciente = 0
destaques = llm_lembrar_medico(ID_paciente)
destaques

'Como médico, não posso esquecer os seguintes pontos ao analisar o caso da Ana Silva:\n\n**Prioridades:**\n\n1. **Toxicidade Ocular:** A visão turva e dificuldade de adaptação à luz relatadas por Ana, em conjunto com o uso de hidroxicloroquina, levantam a suspeita de **toxicidade ocular**.  Este é um efeito colateral grave e potencialmente irreversível da hidroxicloroquina. Uma **avaliação oftalmológica urgente** é crucial para confirmar o diagnóstico e determinar a conduta adequada, que pode incluir a suspensão do medicamento.\n2. **Hepatotoxicidade:** A elevação de AST e ALT sugere **hepatotoxicidade**, um efeito colateral conhecido do metotrexato. É fundamental investigar a causa dessa elevação, considerando outros fatores como uso de álcool, outras medicações e histórico familiar. A relação risco-benefício do metotrexato deve ser reavaliada, e a **suspensão temporária** do medicamento pode ser necessária até a normalização das enzimas hepáticas.\n3. **Anemia:** A hemoglobina baixa 

In [ ]:
ID_paciente = 0
destaques = llm_lembrar_paciente(ID_paciente)
destaques

'Ana, fico feliz em saber que você está sentindo melhora nos sintomas da artrite reumatoide com o tratamento. No entanto, os efeitos colaterais que você está experienciando são preocupantes e precisamos investigá-los.\n\n**O que você não pode esquecer:**\n\n1. **Consulta com Oftalmologista:** A visão turva e a dificuldade de adaptação à luz podem ser efeitos colaterais da hidroxicloroquina. Você precisa **agendar uma consulta com um oftalmologista com urgência** para um exame completo e avaliar a possibilidade de toxicidade ocular.\n2. **Monitorar a Pressão Arterial:** O aumento da pressão arterial pode estar relacionado ao uso da prednisona. É importante **monitorar sua pressão arterial regularmente** em casa e informar qualquer alteração significativa.\n3. **Controlar o Peso:** O ganho de peso também pode ser um efeito colateral da prednisona. É importante **manter uma dieta balanceada e praticar exercícios físicos** para controlar o peso.\n4. **Investigar a Anemia:** A diminuição da

# Cruzando informacoes dos pacientes

In [ ]:
query = "Pacientes que se queixam de dores"
buscar_info()
